In [ ]:
import os
os.environ['R_HOME'] = "C:/Program Files/R/R-4.4.1"  # Adjust if needed
os.environ['R_USER'] = "C:/Users/hhp1483/Documents/R/win-library/4.4"

import rpy2.robjects as ro
print(ro.r('version'))  # Should now show R 4.4.1


### **Return Period Analysis and Data Preparation**
#### **Overview**
This step initializes the return period analysis by importing necessary libraries and defining the paths for the **Annual Maximum Series (AMS)** datasets. The AMS data includes **historical observations** and **future projections** under two climate scenarios (SSP2-4.5 and SSP5-8.5). 

The framework ensures that all datasets are correctly formatted and available for further statistical analysis.



#### **Steps Performed**
1. **Import Required Libraries**
   - **rpy2 & lmom R Package:** For L-moment calculations and probability distribution fitting.

2. **Define Paths for AMS Data**
   - Specifies the **file paths** for:
     - **Observed AMS data**
     - **Future AMS projections** under **SSP2-4.5 and SSP5-8.5**
   - Uses a **structured directory format** to maintain consistency in data storage.

3. **Scenario Mapping**
   - Maps the climate scenarios **(SSP2-4.5 → `ssp245`, SSP5-8.5 → `ssp585`)** to their corresponding **model filenames**.

#### **Instructions for Use**
- Ensure that all AMS CSV files exist in the **specified directory (`E:/Projection_SWAT/ScenarioSs_SWAT_/AMS_Final/`)**.
- Verify that the file naming conventions are **consistent** with the script.
- The observed and future AMS datasets should be structured with **columns labeled "Year" and "AMS"**.



#### **Expected Output**
- **Correctly loaded AMS datasets** for both historical observations and future projections.
- **Scenario mapping completed**, linking models to the appropriate SSP climate scenario.


In [2]:
import pandas as pd
import numpy as np
from scipy import stats
from scipy.stats import pearson3 as pe3
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from glob import glob
import os
import rpy2
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri
from rpy2.robjects import default_converter
from rpy2.robjects.conversion import localconverter
from rpy2.robjects.packages import importr
rlmom = importr('lmom')

path = r'D:/CMIP6-BiasCorrection-SWAT/workingfolder/AMS'
pattern = os.path.join(path, '*.csv')
csv_files = glob(pattern)

print(csv_files)


### **Fitting Probability Distributions to Historical AMS Data (21th)**
#### **Overview**
This step applies **L-Moments methods** to the **observed Annual Maximum Series (AMS) dataset** to determine the best-fitting probability distribution. Several candidate distributions are tested, including **Log-Pearson III (LP3), Generalized Extreme Value (GEV), Pearson III (PE3), Log-Normal (LN3), Generalized Pareto (GPA), and Gumbel (GUM)**.

The return period curves for each distribution are plotted to visually compare their fits, helping us identify the most suitable distribution for historical streamflow data.



#### **Steps Performed**
1. **Load Historical AMS Data**
   - Reads the **observed AMS dataset** from the defined CSV file.
   - Converts the **"Year"** column to a datetime format and ensures correct indexing.

2. **Compute L-Moments**
   - Uses the **lmoments R package** via `rpy2` to compute L-Moments for the observed AMS data.
   - Applies the **L-Moments method** to estimate distribution parameters.

3. **Fit Multiple Probability Distributions**
   - Fits the following **six probability distributions** using L-Moments:
     - **Log-Pearson III (LP3)**
     - **Generalized Extreme Value (GEV)**
     - **Pearson Type III (PE3)**
     - **Log-Normal (LN3)**
     - **Generalized Pareto (GPA)**
     - **Gumbel (GUM)**
   - Extracts distribution parameters and computes return period estimates.

4. **Plot Return Period Curves**
   - Creates a **log-scale return period plot** comparing all distributions.
   - Includes **5-year to 200-year return periods**.
   - Highlights observed AMS data points.

5. **Determine the Best-Fitting Distribution**
   - Compares the **fitted return period curves** to observed data.
   - Identifies the best-fit distribution based on **visual agreement** and **statistical accuracy**.



#### **Instructions for Use**
- Ensure that the **lmoments R package** is installed and accessible via `rpy2`.
- Verify that the observed AMS data is correctly formatted (`"Year"` column as datetime and `"AMS"` as float values).
- The generated plot will visually compare different distributions for further analysis.



#### **Expected Output**
- **L-Moments parameter estimates** for each tested distribution.
- **A return period curve plot** displaying all fitted distributions alongside observed AMS data.
- **Identification of the best-fitting distribution** for historical streamflow analysis.


### L-Moments (LogPearson-III)

In [ ]:
# Load Observed AMS Data
ams_hist = pd.read_csv(f"{path}/observed_ams.csv")
ams_hist = ams_hist['AMS'].values.reshape(-1, 1)

# Create an Empty Array to Store Results
results = np.empty((1, 9))

# Log-Transform the AMS Data for LP-III Fitting
sampling = np.log10(ams_hist)

# Define the Return Periods (5, 10, 25, 50, 100, 200 years)
F = np.array([1 - (1/x) for x in [5, 10, 25, 50, 100, 200]])

# Compute L-Moments Using R's `lmom` Package
mom = rlmom.samlmu(ro.FloatVector(sampling))  # Sample L-moments
par = rlmom.pelpe3(mom)  # Fit LP-III Parameters

# Compute Return Values Using LP-III Distribution
returns = rlmom.quape3(ro.FloatVector(F), par)

# Store Results in Array
results[0, :6] = 10 ** np.array(returns)  # Convert back from log-space
results[0, 6:] = par  # Store LP-III parameters

# Save Results to a `.npy` File
os.makedirs('./results', exist_ok=True)  # Ensure the directory exists
with open('./results/hist-lpe3.npy', 'wb') as f:
    np.save(f, results)

print("LP-III distribution for observed AMS processed and saved successfully.")


### L-Moments (PE3)

In [ ]:
# Create an empty array to store results
results = np.empty((1, 9))

# Use the original AMS values (without log transformation)
sampling = np.array(ams_hist)

# Define the Return Periods (5, 10, 25, 50, 100, 200 years)
F = np.array([1 - (1/x) for x in [5, 10, 25, 50, 100, 200]])

# Compute L-Moments Using R's `lmom` Package
mom = rlmom.samlmu(ro.FloatVector(sampling))  # Sample L-moments
par = rlmom.pelpe3(mom)  # Fit PE3 Parameters

# Compute Return Values Using PE3 Distribution
returns = rlmom.quape3(ro.FloatVector(F), par)

# Store Results in Array
results[0, :6] = np.array(returns)  # Store return levels
results[0, 6:] = par  # Store PE3 parameters

# Ensure the results directory exists
os.makedirs('./results', exist_ok=True)

# Save Results to a `.npy` File
with open('./results/hist-pe3.npy', 'wb') as f:
    np.save(f, results)

print("PE3 distribution for observed AMS processed and saved successfully.")


### L-Moments (GEV)

In [ ]:
# Create an empty array to store results
results = np.empty((1, 9))

# Use the original AMS values (without log transformation)
sampling = np.array(ams_hist)

# Define the Return Periods (5, 10, 25, 50, 100, 200 years)
F = np.array([1 - (1/x) for x in [5, 10, 25, 50, 100, 200]])

# Compute L-Moments Using R's `lmom` Package
mom = rlmom.samlmu(ro.FloatVector(sampling))  # Sample L-moments
par = rlmom.pelgev(mom)  # Fit GEV Parameters

# Compute Return Values Using GEV Distribution
returns = rlmom.quagev(ro.FloatVector(F), par)

# Store Results in Array
results[0, :6] = np.array(returns)  # Store return levels
results[0, 6:] = par  # Store GEV parameters

# Ensure the results directory exists
os.makedirs('./results', exist_ok=True)

# Save Results to a `.npy` File
with open('./results/hist-gev.npy', 'wb') as f:
    np.save(f, results)

print("GEV distribution for observed AMS processed and saved successfully.")


### L-Moments (LN3)

In [ ]:
# Create an empty array to store results
results = np.empty((1, 9))

# Log-transform AMS data for LN3 fitting
sampling = np.log(np.array(ams_hist))

# Define the Return Periods (5, 10, 25, 50, 100, 200 years)
F = np.array([1 - (1/x) for x in [5, 10, 25, 50, 100, 200]])

# Compute L-Moments Using R's `lmom` Package
mom = rlmom.samlmu(ro.FloatVector(sampling))  # Sample L-moments
par = rlmom.pelln3(mom, bound=0)  # Fit LN3 Parameters

# Compute Return Values Using LN3 Distribution
returns = rlmom.qualn3(ro.FloatVector(F), par)

# Store Results in Array
results[0, :6] = np.exp(np.array(returns))  # Convert back from log-space
results[0, 6:] = par  # Store LN3 parameters

# Ensure the results directory exists
os.makedirs('./results', exist_ok=True)

# Save Results to a `.npy` File
with open('./results/hist-ln3.npy', 'wb') as f:
    np.save(f, results)

print("LN3 distribution for observed AMS processed and saved successfully.")


### L-Moments (GPA)

In [ ]:
# Create an empty array to store results
results = np.empty((1, 9))

# Use the original AMS values (without log transformation)
sampling = np.array(ams_hist)

# Define the Return Periods (5, 10, 25, 50, 100, 200 years)
F = np.array([1 - (1/x) for x in [5, 10, 25, 50, 100, 200]])

# Compute L-Moments Using R's `lmom` Package
mom = rlmom.samlmu(ro.FloatVector(sampling))  # Sample L-moments
par = rlmom.pelgpa(mom)  # Fit GPA Parameters

# Compute Return Values Using GPA Distribution
returns = rlmom.quagpa(ro.FloatVector(F), par)

# Store Results in Array
results[0, :6] = np.array(returns)  # Store return levels
results[0, 6:] = par  # Store GPA parameters

# Ensure the results directory exists
os.makedirs('./results', exist_ok=True)

# Save Results to a `.npy` File
with open('./results/hist-gpa.npy', 'wb') as f:
    np.save(f, results)

print("GPA distribution for observed AMS processed and saved successfully.")


### L-Moments (Gumbel)

In [ ]:
# Create an empty array to store results
results = np.empty((1, 8))

# Use the original AMS values (without log transformation)
sampling = np.array(ams_hist)

# Define the Return Periods (5, 10, 25, 50, 100, 200 years)
F = np.array([1 - (1/x) for x in [5, 10, 25, 50, 100, 200]])

# Compute L-Moments Using R's `lmom` Package
mom = rlmom.samlmu(ro.FloatVector(sampling))  # Sample L-moments
par = rlmom.pelgum(mom)  # Fit Gumbel (GUM) Parameters

# Compute Return Values Using Gumbel Distribution
returns = rlmom.quagum(ro.FloatVector(F), par)

# Store Results in Array
results[0, :6] = np.array(returns)  # Store return levels
results[0, 6:] = par  # Store Gumbel parameters

# Ensure the results directory exists
os.makedirs('./results', exist_ok=True)

# Save Results to a `.npy` File
with open('./results/hist-gum.npy', 'wb') as f:
    np.save(f, results)

print("Gumbel distribution for observed AMS processed and saved successfully.")


### Plot Returns

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Font and figure settings
XSMALL_SIZE = 8
SMALL_SIZE = 10
MEDIUM_SIZE = 12
BIGGER_SIZE = 14

plt.rc('font', size=SMALL_SIZE)
plt.rc('axes', titlesize=BIGGER_SIZE, labelsize=MEDIUM_SIZE)
plt.rc('xtick', labelsize=SMALL_SIZE)
plt.rc('ytick', labelsize=SMALL_SIZE)
plt.rc('legend', fontsize=SMALL_SIZE)
plt.rc('figure', titlesize=BIGGER_SIZE, facecolor='white', dpi=300)

# Create a 1-row, 2-column subplot
fig, axes = plt.subplots(ncols=2, nrows=1, dpi=300, figsize=(5 * 5, 4.3 * 2))
fig.subplots_adjust(hspace=0.3, wspace=0.2)  # Adjust spacing between plots

# Scenario labels
scn_labels = ['SSP2', 'SSP5']
scenarios = ['4.5', '8.5']

# Load the results from the saved .npy files
hist_lpe3 = np.load('./results/hist-lpe3.npy')
hist_pe3 = np.load('./results/hist-pe3.npy')
hist_gev = np.load('./results/hist-gev.npy')
hist_ln3 = np.load('./results/hist-ln3.npy')
hist_gpa = np.load('./results/hist-gpa.npy')
hist_gum = np.load('./results/hist-gum.npy')

# Find the min and max values across all datasets for consistent y-axis scaling
ymin = np.min([np.min(hist_lpe3[:, :6]), np.min(hist_pe3[:, :6]), np.min(hist_gev[:, :6]), 
               np.min(hist_ln3[:, :6]), np.min(hist_gpa[:, :6]), np.min(hist_gum[:, :6])])
ymax = np.max([np.max(hist_lpe3[:, :6]), np.max(hist_pe3[:, :6]), np.max(hist_gev[:, :6]), 
               np.max(hist_ln3[:, :6]), np.max(hist_gpa[:, :6]), np.max(hist_gum[:, :6])])

# Loop through each scenario and plot the data
for sc_i, sc in enumerate(scenarios):
    ax = axes[sc_i]

    ax.plot([5,10,25,50,100,200], hist_lpe3[0, :6], color='red', linestyle='-', linewidth=3, label='Observed Data (LogP-III)')
    ax.plot([5,10,25,50,100,200], hist_pe3[0, :6], color='orange', linestyle='dotted', linewidth=3, label='Observed Data (P-III)')
    ax.plot([5,10,25,50,100,200], hist_gev[0, :6], color='blue', linestyle='-', linewidth=3, label='Observed Data (GEV)')
    ax.plot([5,10,25,50,100,200], hist_ln3[0, :6], color='green', linestyle='-', linewidth=3, label='Observed Data (LN3)')
    ax.plot([5,10,25,50,100,200], hist_gpa[0, :6], color='black', linestyle='dotted', linewidth=3, label='Observed Data (GPA)')
    ax.plot([5,10,25,50,100,200], hist_gum[0, :6], color='magenta', linestyle='-', linewidth=3, label='Observed Data (Gumbel)')
    
    ax.set_title('Frequency analysis\nObserved Data (1984-2016)')
    ax.legend(loc='upper left')
    ax.set_xscale('log')  # Set x-axis to logarithmic scale
    ax.set_xticks([5, 10, 25, 50, 100, 200])
    ax.set_xticklabels([5, 10, 25, 50, 100, 200])  # Set labels for xticks
    ax.set_xlim((5, 200))
    ax.yaxis.set_major_formatter(plt.matplotlib.ticker.StrMethodFormatter('{x:,.0f}'))
    
    if sc_i == 0:
        ax.set_ylabel('streamflow [$m^3/s$]')
        
    ax.set_xlabel('Return Period [years]')
    ax.grid(True, which="both", linestyle="--", alpha=0.5)
    ax.set_ylim((ymin, ymax))

# Ensure results directory exists before saving
import os
os.makedirs('./results', exist_ok=True)

# Save the figure
plt.savefig('./results/return_periods_hist.jpg', bbox_inches='tight', dpi=300)
plt.show()


### **Applying the Best-Fit Distribution to Future AMS Projections**
#### **Overview**
Once the **best-fitting probability distribution** is identified from the historical AMS data, it is applied to **future climate projections** under the **SSP2-4.5 and SSP5-8.5 scenarios**. This step ensures that the selected distribution is used consistently across both historical and future datasets.

A **random sampling approach** is employed to **account for model uncertainty** in future AMS projections, ensuring robust return period estimates.


#### **Steps Performed**
1. **Apply the Best-Fit Distribution to Historical AMS**
   - Uses the previously identified **best-fit distribution**.
   - Computes return period estimates for the **historical dataset**.
   - Ensures consistency between historical and future analysis.

2. **Load and Preprocess Future AMS Projections**
   - Reads **future AMS datasets** for the **SSP2-4.5 and SSP5-8.5 scenarios**.
   - Converts **"Year"** column to datetime format and sets it as an index.
   - Extracts AMS values from **multiple climate models**.

3. **Perform Random Sampling for Uncertainty Estimation**
   - Uses **random sampling from three models** per scenario.
   - Generates **1000 bootstrap samples** per climate scenario.
   - Fits the **best-fit distribution** to each sample.

4. **Compute Return Period Estimates for Future Scenarios**
   - Computes **5th, 50th (median), and 95th percentile** return periods for each future scenario.
   - Ensures that projections align with **historical trends**.

5. **Save Results for Visualization and Further Analysis**
   - Stores computed return period estimates for each scenario.
   - Saves results in a structured format for **plotting and comparison**.



#### **Instructions for Use**
- Ensure that the best-fit distribution identified from historical AMS data is correctly applied.
- Verify that future AMS datasets follow the **same structure as historical data**.
- Modify the number of bootstrap samples (`1000` by default) if needed.



#### **Expected Output**
- **Future AMS return period estimates** using the best-fit distribution.
- **Bootstrap samples** for model uncertainty analysis.
- **Statistical outputs stored for visualization** in the next step.


### L-Moments (LogPearson-III) with random sampling

In [ ]:
import os
import numpy as np
import pandas as pd
import rpy2.robjects as ro
from rpy2.robjects.packages import importr

# Load R's lmom package
rlmom = importr('lmom')

# Define the base path where the CSV files are stored
path = r'E:/Projection_SWAT/ScenarioSs_SWAT_/AMS_Final'

# Define scenario mappings to match filenames
scenarios = {'4.5': 'ssp245', '8.5': 'ssp585'}

for sc_i, (sc, ssp) in enumerate(scenarios.items()):
    # Read files and set 'Year' column as index
    access = pd.read_csv(f"{path}/ACCESS-ESM1-5_{ssp}.csv")
    access['Year'] = pd.to_datetime(access['Year'], format='%Y')
    access = access.set_index('Year')

    if sc == '4.5':
        mpi_esmi = pd.read_csv(f"{path}/MPI-ESM1-2-HR_{ssp}.csv")
        mpi_esmi['Year'] = pd.to_datetime(mpi_esmi['Year'], format='%Y')
        mpi_esmi = mpi_esmi.set_index('Year')
    else:
        giss = pd.read_csv(f"{path}/GISS-E2-1-G_{ssp}.csv")
        giss['Year'] = pd.to_datetime(giss['Year'], format='%Y')
        giss = giss.set_index('Year')

    cnrm = pd.read_csv(f"{path}/CNRM-ESM2-1_{ssp}.csv")
    cnrm['Year'] = pd.to_datetime(cnrm['Year'], format='%Y')
    cnrm = cnrm.set_index('Year')

    if sc == '4.5':
        print(sc, access.columns, mpi_esmi.columns, cnrm.columns)
        ams_pool = access[['AMS']].join(mpi_esmi[['AMS']], rsuffix='_mpi_esmi').join(cnrm[['AMS']], rsuffix='_cnrm')
    else:
        print(sc, access.columns, giss.columns, cnrm.columns)
        ams_pool = access[['AMS']].join(giss[['AMS']], rsuffix='_giss').join(cnrm[['AMS']], rsuffix='_cnrm')

    # Reshape dataset to (75, 3) for random sampling
    ams_pool = ams_pool.values.reshape(75, 3)

    # Create empty array for results (1000 simulations x 9 columns)
    results = np.empty((1000, 9))

    # Set up random number generator
    rng = np.random.default_rng(int('10%03d' % sc_i))  # Better randomness control

    # Perform 1000 iterations for random sampling
    for sim in range(1000):
        # Generate random indices for sampling 75 values from 3 models
        indices = rng.integers(low=0, high=3, size=75)

        # Extract randomly sampled AMS values
        sampling = np.log10(np.array([ams_pool[i, model] for i, model in enumerate(indices)]))

        # Define return periods
        F = np.array([1 - (1/x) for x in [5, 10, 25, 50, 100, 200]])

        # Compute L-Moments and fit Log-Pearson III (LP-III) parameters
        mom = rlmom.samlmu(ro.FloatVector(sampling))
        par = rlmom.pelpe3(mom)
        returns = rlmom.quape3(ro.FloatVector(F), par)

        # Store results
        results[sim, :6] = 10 ** np.array(returns)  # Convert back from log-space
        results[sim, 6:] = par  # Store LP-III parameters

    # Ensure results directory exists
    os.makedirs('./results', exist_ok=True)

    # Save results to `.npy` file
    with open(f'./results/{sc}-fut-sim-lpe3.npy', 'wb') as f:
        np.save(f, results)

    print(f"Scenario {sc}: Future simulations processed and saved successfully.")


#### Plot Returns

In [ ]:
# Font and figure settings
XSMALL_SIZE = 8
SMALL_SIZE = 10
MEDIUM_SIZE = 17
BIGGER_SIZE = 24

plt.rc('font', size=BIGGER_SIZE)
plt.rc('axes', titlesize=BIGGER_SIZE, labelsize=BIGGER_SIZE)
plt.rc('xtick', labelsize=MEDIUM_SIZE)
plt.rc('ytick', labelsize=MEDIUM_SIZE)
plt.rc('legend', fontsize=MEDIUM_SIZE)
plt.rc('figure', titlesize=BIGGER_SIZE, facecolor='white', dpi=300)

# Create a 1-row, 2-column subplot
fig, axes = plt.subplots(ncols=2, nrows=1, dpi=300, figsize=(5 * 5, 4.3 * 2))
plt.subplots_adjust(hspace=0.5, wspace=0.15)

# Load historical observed data
hist_lpe3 = np.load('./results/hist-lpe3.npy')

# Define scenario labels and mappings
scn_labels = ['SSP2', 'SSP5']
scenarios = ['4.5', '8.5']

# Loop through each scenario and plot the simulations
for sc_i, sc in enumerate(scenarios):
    
    # Load future simulations
    fut = np.load(f'./results/{sc}-fut-sim-lpe3.npy')  # Corrected file extension to `.npy`
    
    # Axis limits
    ymin = 10000
    ymax = np.max(fut[:, :6])
    
    simulations = fut
    
    ax = axes[sc_i]
    
    # Plot all simulations with transparency
    for sim in simulations:
        ax.plot([5,10,25,50,100,200], sim[:6], color='#889', alpha=0.1)
    
    # Plot the percentile-based confidence bounds
    ax.plot([5,10,25,50,100,200], np.percentile(simulations[:, :6], 5, axis=0), color='blue', linestyle='--', linewidth=2, label='21st Century Low (5th)')
    ax.plot([5,10,25,50,100,200], np.median(simulations[:, :6], axis=0), color='lightgreen', linestyle='--', linewidth=2, label='21st Century Median (50th)')
    ax.plot([5,10,25,50,100,200], np.percentile(simulations[:, :6], 95, axis=0), color='red', linestyle='--', linewidth=2, label='21st Century High (95th)')
    
    # Plot observed historical data
    ax.plot([5,10,25,50,100,200], hist_lpe3[0, :6], color='fuchsia', linestyle='-', linewidth=2, label='20th Century')
    
    ax.legend(loc='upper left')
    
    # Logarithmic x-axis
    ax.set_xscale('log')
    ax.set_xticks([5,10,25,50,100,200])
    ax.set_xticklabels([5,10,25,50,100,200])
    ax.set_xlim((5, 200))
    
    # Format y-axis
    ax.yaxis.set_major_formatter(plt.matplotlib.ticker.StrMethodFormatter('{x:,.0f}'))
    
    if sc_i == 0:
        ax.set_ylabel(r'Streamflow ($\bf{m^3/s}$)', weight='bold')
    
    ax.set_xlabel('Return Period (Years)', weight='bold')
    ax.grid()
    ax.set_ylim((ymin, ymax))
    
    # Add labels 'a' and 'b' inside each subplot
    if sc_i == 0:
        ax.text(0.95, 0.95, 'a', transform=ax.transAxes, fontsize=36, ha='center', va='center', weight='bold')
    elif sc_i == 1:
        ax.text(0.95, 0.95, 'b', transform=ax.transAxes, fontsize=36, ha='center', va='center', weight='bold')

# Ensure results directory exists before saving
import os
os.makedirs('./results', exist_ok=True)

# Save the figure as a high-resolution SVG
plt.savefig(r'D:/CMIP6-BiasCorrection-SWAT/workingfolder/Results_Plots/flow-grids-lpe3-Fig5.png', format="png", bbox_inches='tight', dpi=600)
# Save the figure as a high-resolution PDF
#plt.savefig(r'D:/CMIP6-BiasCorrection-SWAT/workingfolder/Results_Plots/flow-grids-lpe3-Fig5.pdf', format="pdf", bbox_inches='tight', dpi=600)

plt.show()
